In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import activations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [ ]:
train = '/kaggle/input/animals-detection-images-dataset/train'
test = '/kaggle/input/animals-detection-images-dataset/test'

In [ ]:
classes = ['Deer','Eagle','Horse','Monkey','Chicken','Lizard']

In [ ]:
train_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
train_images = train_generator.flow_from_directory(
    train,
    target_size=(32, 32),
    batch_size=64,
    class_mode='categorical',
    classes=classes)

In [ ]:
test_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
test_images = test_generator.flow_from_directory(
    test,
    target_size=(32, 32),
    batch_size=64,
    class_mode='categorical',
    classes=classes)

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(classes), activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_images, epochs=10, validation_data=test_images)

In [ ]:
test_images = []
for animal in classes:
    animal_file = os.path.join(test, animal)
    for i in range(1):
        image_file = random.choice(os.listdir(animal_file))
        image_path = os.path.join(animal_file, image_file)
        test_images.append((image_path, animal))

for image_path, predicted_animal in test_images:

    image = load_img(image_path, target_size=(32, 32))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)

    predictions = model.predict(image)
    predicted_animal = classes[np.argmax(predictions)]
    
    plt.axis('off')
    plt.imshow(load_img(image_path))
    plt.title(predicted_animal)
    plt.show()